In [1]:
import optuna
from optuna.samplers import TPESampler
import warnings

import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import math
from collections import defaultdict
import seaborn as sns
from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import Lasso, Ridge, RidgeCV
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    make_scorer,
    precision_score,
    recall_score,
    roc_auc_score,
)
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from xgboost import XGBClassifier
from pathlib import Path

warnings.filterwarnings("ignore")

## 🏗️ Building Final Datasets from Summary Files

### 📌 HOSP dataset
- **Includes:** all patients (`patients.csv`) + `labevents_summary` + `OMR` values  
- **Filling strategy:** missing OMR filled with `charteevents` where available  
- **Coverage:** keeps as many subjects as possible (**≈36,899**)  

---

### 📌 ICU+HOSP dataset
- **Includes:** only patients present in `charteevents` (ICU)  
- **Joined with:** `labevents_summary` and OMR where possible  
- **Coverage:** smaller but richer cohort (**≈11k**)  

---

### ⚙️ Code pipeline performs:
- ✅ Standardizes column names  
- ✅ Avoids duplicate / redundant columns  
- ✅ Fills OMR missingness from `charteevents`  
- ✅ Drops extremely sparse columns (configurable threshold)  
- ✅ Creates indicator flags:  
  - `has_labs`  
  - `has_omr`  
  - `has_charte`  
- ✅ Imputes remaining numeric missing values (median, configurable)  
- ✅ Saves outputs in **CSV** and **Parquet**  

---


In [28]:
# ---------- CONFIG ----------
DATA_DIR = Path(".")  # change to folder where CSVs are
patients_fp = DATA_DIR / "patients.csv"
labs_fp = DATA_DIR / "labevents_summary.csv"
omr_fp = DATA_DIR / "omr_summary.csv"
charte_fp = DATA_DIR / "charteevents_subject_summary.csv"

OUT_HOSP_CSV = DATA_DIR / "final_hosp_dataset.csv"
OUT_HOSP_PARQ = DATA_DIR / "final_hosp_dataset.parquet"
OUT_ICU_CSV  = DATA_DIR / "final_icu_hosp_dataset.csv"
OUT_ICU_PARQ = DATA_DIR / "final_icu_hosp_dataset.parquet"

# thresholds / choices:
SPARSITY_THRESHOLD = 0.80   # drop columns with >80% missing
IMPUTE_WITH_MEDIAN = False
# ----------------------------

# 1) Load summary tables
print("Loading summaries...")
patients = pd.read_csv(patients_fp)   # ['subject_id', 'gender', 'anchor_age', 'label']
labs = pd.read_csv(labs_fp)           # labevents_summary (subject_id + *_mean)
omr = pd.read_csv(omr_fp)             # omr_summary
charte = pd.read_csv(charte_fp)       # chartevents_subject_summary

# 2) Normalize column names (make them consistent and short)
def clean_cols(df):
    df = df.copy()
    df.columns = [c.strip().replace(" ", "_").replace("(", "").replace(")", "").replace("-", "_").replace(".", "").replace("/", "_") for c in df.columns]
    return df

patients = clean_cols(patients)
labs = clean_cols(labs)
omr = clean_cols(omr)
charte = clean_cols(charte)

# Example mapping for charte to align names with omr
# charte column examples: Heart_Rate_mean, Non_Invasive_Blood_Pressure_systolic_mean, ...
# omr columns: bp_sys_mean, bp_dia_mean, weight_kg_mean, height_cm_mean
charte_rename = {
    "Heart_Rate_mean": "hr_mean",
    "Non_Invasive_Blood_Pressure_systolic_mean": "bp_sys_charte_mean",
    "Non_Invasive_Blood_Pressure_diastolic_mean": "bp_dia_charte_mean",
    "Respiratory_Rate_mean": "rr_mean",
    "Temperature_Celsius_mean": "tempC_charte_mean",
    "Admission_Weight_Kg_mean": "weight_charte_mean",
    "Height_cm_mean": "height_charte_mean"
}
# apply mapping only for columns that exist
charte = charte.rename(columns={k:v for k,v in charte_rename.items() if k in charte.columns})

# Omr column names expected: weight_kg_mean, height_cm_mean, bmi_mean, bp_sys_mean, bp_dia_mean, egfr_mean
# Labs already have appropriate *_mean names (like Creatinine_mean etc.)

# 3) Build HOSP dataset: patients LEFT JOIN labs LEFT JOIN omr
print("Merging patients + labs + omr for HOSP dataset...")
hosp = patients.merge(labs, on="subject_id", how="left")
hosp = hosp.merge(omr, on="subject_id", how="left")

# 4) Use chartevents to fill missing OMR values (weight/height/bp) where available:
# columns to try to fill: weight_kg_mean, height_cm_mean, bp_sys_mean, bp_dia_mean
fill_map = {
    "weight_kg_mean": "weight_charte_mean",
    "height_cm_mean": "height_charte_mean",
    "bp_sys_mean": "bp_sys_charte_mean",
    "bp_dia_mean": "bp_dia_charte_mean"
}

# If the omr column doesn't exist (maybe named differently), adjust accordingly
for omr_col, char_col in fill_map.items():
    if omr_col not in hosp.columns and char_col in charte.columns:
        # nothing to fill if omr col absent; skip
        continue
    if char_col in charte.columns:
        # create a mapping from charte df
        char_map = charte.set_index("subject_id")[char_col]
        # fill only where hosp[omr_col] is missing
        hosp[omr_col] = hosp[omr_col].fillna(hosp["subject_id"].map(char_map))

# 5) Create indicator flags for presence of data
hosp["has_labs"] = hosp[[c for c in labs.columns if c.endswith("_mean") and c!="subject_id"]].notna().any(axis=1).astype(int)
omr_mean_cols = [c for c in hosp.columns if c.endswith("_mean") and c.startswith(("weight","height","bp","bmi","egfr"))]
hosp["has_omr"] = hosp[omr_mean_cols].notna().any(axis=1).astype(int)

# Also create has_charte flag by checking if subject in charte
charte_subjects = set(charte["subject_id"].unique())
hosp["has_charte"] = hosp["subject_id"].isin(charte_subjects).astype(int)

# 6) Decide which columns are redundant / to keep
# Keep: demographics, label, labs *_mean, omr relevant *_mean (weight,height,bp,bmi), hr/rr maybe from charte if no omr
keep_cols = ["subject_id", "gender", "anchor_age", "label", "has_labs", "has_omr", "has_charte"]

# add lab mean columns
lab_mean_cols = [c for c in labs.columns if c.endswith("_mean") and c!="subject_id"]
keep_cols += lab_mean_cols

# prefer OMR columns (if present); if not, include charte equivalents to fill
preferred_omr = ["weight_kg_mean", "height_cm_mean", "bmi_mean", "bp_sys_mean", "bp_dia_mean"]
for c in preferred_omr:
    if c in hosp.columns:
        keep_cols.append(c)
# also include HR/RR/temp from charte if present
for c in ["hr_mean","rr_mean","tempC_charte_mean"]:
    if c in charte.columns:
        # but hosp doesn't yet contain them; map charte values to hosp
        hosp[c] = hosp["subject_id"].map(charte.set_index("subject_id")[c]) if c in charte.columns else np.nan
        keep_cols.append(c)

# ensure uniqueness and existence
keep_cols = [c for c in keep_cols if c in hosp.columns]
print("Columns kept for HOSP dataset:", keep_cols)

hosp_final = hosp[keep_cols].copy()

# 7) Drop excessively sparse columns (optional) in hosp_final
missing_frac = hosp_final.isna().mean().sort_values(ascending=False)
cols_to_drop = missing_frac[missing_frac > SPARSITY_THRESHOLD].index.tolist()
print("Dropping columns with >{:.0%} missing: {}".format(SPARSITY_THRESHOLD, cols_to_drop))
hosp_final = hosp_final.drop(columns=cols_to_drop)

# 8) Impute numeric missing values (simple median) and keep has_omr/has_labs flags
num_cols = hosp_final.select_dtypes(include=[np.number]).columns.tolist()
# exclude label if numeric and you don't want to impute it
if "label" in num_cols:
    num_cols.remove("label")
if IMPUTE_WITH_MEDIAN:
    for c in num_cols:
        med = hosp_final[c].median(skipna=True)
        hosp_final[c] = hosp_final[c].fillna(med)


# ---------- Build ICU+HOSP enriched dataset ----------
print("Building ICU+HOSP dataset (subjects present in charteevents)...")
charte_subjects_list = sorted(list(charte_subjects))
icu = patients[patients["subject_id"].isin(charte_subjects_list)].copy()

# merge labs and omr (omr may be missing for many)
icu = icu.merge(labs, on="subject_id", how="left")
icu = icu.merge(omr, on="subject_id", how="left")
# merge charte features directly (charte already has mean stats)
icu = icu.merge(charte.set_index("subject_id"), on="subject_id", how="left")

# Build keep list for ICU: demographics + lab means + charte vitals + omr where available
icu_keep = ["subject_id", "gender", "anchor_age", "label"]
icu_keep += lab_mean_cols
# add charte vitals (renamed earlier): hr_mean, rr_mean, tempC_charte_mean, weight_charte_mean, height_charte_mean
for c in ["hr_mean","rr_mean","tempC_charte_mean","weight_charte_mean","height_charte_mean"]:
    if c in icu.columns:
        icu_keep.append(c)
# optionally include omr BMI if present
if "bmi_mean" in icu.columns:
    icu_keep.append("bmi_mean")

icu_keep = [c for c in icu_keep if c in icu.columns]
icu_final = icu[icu_keep].copy()

# Drop super-sparse columns
missing_frac_icu = icu_final.isna().mean().sort_values(ascending=False)
cols_to_drop_icu = missing_frac_icu[missing_frac_icu > SPARSITY_THRESHOLD].index.tolist()
icu_final = icu_final.drop(columns=cols_to_drop_icu)

# Impute numeric missing with median (except label)
num_cols_icu = icu_final.select_dtypes(include=[np.number]).columns.tolist()
if "label" in num_cols_icu:
    num_cols_icu.remove("label")
if IMPUTE_WITH_MEDIAN:
    for c in num_cols_icu:
        med = icu_final[c].median(skipna=True)
        icu_final[c] = icu_final[c].fillna(med)


# 10) final diagnostics
print("HOSP subjects:", hosp_final['subject_id'].nunique(), "rows:", hosp_final.shape)
print("ICU+HOSP subjects:", icu_final['subject_id'].nunique(), "rows:", icu_final.shape)
print("Done.")


Loading summaries...
Merging patients + labs + omr for HOSP dataset...
Columns kept for HOSP dataset: ['subject_id', 'gender', 'anchor_age', 'label', 'has_labs', 'has_omr', 'has_charte', 'Anion_Gap_mean', 'Bicarbonate_mean', 'Chloride_mean', 'Creatinine_mean', 'Glucose_mean', 'Potassium_mean', 'Sodium_mean', 'Urea_Nitrogen_mean', 'weight_kg_mean', 'height_cm_mean', 'bmi_mean', 'bp_sys_mean', 'bp_dia_mean', 'hr_mean', 'rr_mean', 'tempC_charte_mean']
Dropping columns with >80% missing: ['tempC_charte_mean']
Building ICU+HOSP dataset (subjects present in charteevents)...
HOSP subjects: 36899 rows: (36899, 22)
ICU+HOSP subjects: 36899 rows: (36899, 16)
Done.


In [29]:
# list charte-only columns you added earlier (example names)
charte_cols_in_hosp = [c for c in hosp_final.columns if c.endswith("_charte_mean") or c in ['hr_mean','rr_mean','tempC_charte_mean']]
print("Charte-derived columns present in hosp_final:", charte_cols_in_hosp)

# drop if you don't want them in hosp_final
hosp_final = hosp_final.drop(columns=charte_cols_in_hosp, errors='ignore')

# verify
print("HOSP final shape after dropping charte-only cols:", hosp_final.shape)

Charte-derived columns present in hosp_final: ['hr_mean', 'rr_mean']
HOSP final shape after dropping charte-only cols: (36899, 20)


In [30]:
# load the summaries again (paths as you used)
charte = pd.read_csv("charteevents_subject_summary.csv")   # might include reindexed rows
patients = pd.read_csv("patients.csv")
labs = pd.read_csv("labevents_summary.csv")
omr = pd.read_csv("omr_summary.csv")

# normalize column names if needed (optional)
charte.columns = [c.strip() for c in charte.columns]
patients.columns = [c.strip() for c in patients.columns]
labs.columns = [c.strip() for c in labs.columns]
omr.columns = [c.strip() for c in omr.columns]

# 1) determine which charte columns actually contain measurements (mean columns)
# adjust names to match your file; example guessed names:
charte_mean_cols = [c for c in charte.columns if c.endswith("_mean") and c != "subject_id"]

# 2) find rows where at least one charte mean is present -> TRUE ICU subjects
charte['has_any_charte'] = charte[charte_mean_cols].notna().any(axis=1)
true_icu_subjects = set(charte.loc[charte['has_any_charte'], 'subject_id'].unique())

print("Total rows in charte file:", len(charte))
print("Unique subject_ids present in charte file:", charte['subject_id'].nunique())
print("Number of TRUE ICU subjects (non-NaN charte):", len(true_icu_subjects))

# 3) rebuild icu_final: keep only patients who are in true_icu_subjects
icu_patients = patients[patients['subject_id'].isin(true_icu_subjects)].copy()
print("Patients filtered to TRUE ICU patients:", icu_patients.shape)

# 4) merge labs, omr, and the charte measurements
icu = icu_patients.merge(labs, on="subject_id", how="left")
icu = icu.merge(omr, on="subject_id", how="left")
# merge only charte columns that contain real data (avoid the reindexed all-subjects rows causing NaNs)
charte_real = charte[charte['has_any_charte']].set_index('subject_id')
icu = icu.merge(charte_real[charte_mean_cols], on="subject_id", how="left")

# 5) verify
print("ICU final shape (rows,cols):", icu.shape)
print("Unique subject_ids in icu_final:", icu['subject_id'].nunique())
print("Preview missing proportions in icu_final:")
icu.isna().mean().sort_values(ascending=False).head(20)


Total rows in charte file: 36899
Unique subject_ids present in charte file: 36899
Number of TRUE ICU subjects (non-NaN charte): 10969
Patients filtered to TRUE ICU patients: (10969, 5)
ICU final shape (rows,cols): (10969, 32)
Unique subject_ids in icu_final: 10969
Preview missing proportions in icu_final:


egfr_mean                                     0.997721
Temperature_Celsius_mean                      0.854864
height_cm_mean                                0.495305
bmi_mean                                      0.471055
Height_(cm)_mean                              0.430851
weight_kg_mean                                0.429027
bp_dia_mean                                   0.387729
bp_sys_mean                                   0.387729
egfr_count                                    0.328380
bp_dia_count                                  0.328380
bp_sys_count                                  0.328380
bmi_count                                     0.328380
weight_kg_count                               0.328380
height_cm_count                               0.328380
Non_Invasive_Blood_Pressure_diastolic_mean    0.008661
Non_Invasive_Blood_Pressure_systolic_mean     0.008661
Glucose_mean                                  0.002644
Bicarbonate_mean                              0.002644
Anion_Gap_

In [47]:
dc = [c for c in icu.columns if c.endswith("_count")]
icu.drop(columns=dc, inplace=True)
icu.drop(columns=["Unnamed: 0"], inplace=True)

In [10]:
# Save HOSP dataset
print("Saving HOSP dataset. shape:", hosp_final.shape)
hosp_final.to_csv(OUT_HOSP_CSV, index=False)
hosp_final.to_parquet(OUT_HOSP_PARQ, index=False)
print("HOSP saved:", OUT_HOSP_CSV, OUT_HOSP_PARQ)

# Save ICU dataset
print("Saving ICU+HOSP dataset. shape:", icu.shape)
icu.to_csv(OUT_ICU_CSV, index=False)
icu.to_parquet(OUT_ICU_PARQ, index=False)
print("ICU+HOSP saved:", OUT_ICU_CSV, OUT_ICU_PARQ)

Saving HOSP dataset. shape: (36899, 20)


NameError: name 'OUT_HOSP_CSV' is not defined

In [36]:
print(hosp_final.shape)
print(hosp_final.columns)
missing_values_prop = hosp_final.isnull().mean()
a = missing_values_prop*100.0
print("missings proportion:\n", a)
hosp_final.head()

(36899, 20)
Index(['subject_id', 'gender', 'anchor_age', 'label', 'has_labs', 'has_omr',
       'has_charte', 'Anion_Gap_mean', 'Bicarbonate_mean', 'Chloride_mean',
       'Creatinine_mean', 'Glucose_mean', 'Potassium_mean', 'Sodium_mean',
       'Urea_Nitrogen_mean', 'weight_kg_mean', 'height_cm_mean', 'bmi_mean',
       'bp_sys_mean', 'bp_dia_mean'],
      dtype='object')
missings proportion:
 subject_id             0.000000
gender                 0.000000
anchor_age             0.000000
label                  0.000000
has_labs               0.000000
has_omr                0.000000
has_charte             0.000000
Anion_Gap_mean         4.647822
Bicarbonate_mean       4.639692
Chloride_mean          4.528578
Creatinine_mean        3.265671
Glucose_mean           3.783300
Potassium_mean         4.479796
Sodium_mean            4.504187
Urea_Nitrogen_mean     3.718258
weight_kg_mean        25.466815
height_cm_mean        36.767934
bmi_mean              41.922545
bp_sys_mean           31.

,subject_id,gender,anchor_age,label,has_labs,has_omr,has_charte,Anion_Gap_mean,Bicarbonate_mean,Chloride_mean,Creatinine_mean,Glucose_mean,Potassium_mean,Sodium_mean,Urea_Nitrogen_mean,weight_kg_mean,height_cm_mean,bmi_mean,bp_sys_mean,bp_dia_mean
0,10000032,F,52,1,1,1,1,11.222222,25.388889,97.047619,0.466667,106.176471,5.100000,128.700000,30.388889,42.231264,152.40000,18.5375,106.166667,64.666667
1,10000068,F,19,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10000084,M,72,0,1,1,1,12.833333,23.166667,100.666667,0.733333,95.833333,4.350000,136.666667,12.666667,77.110703,177.80000,24.4000,NaN,NaN
3,10000108,M,25,0,1,0,1,15.000000,29.000000,101.000000,1.000000,88.000000,3.500000,141.000000,12.000000,NaN,NaN,NaN,NaN,NaN
4,10000117,F,48,0,1,1,1,16.928571,23.714286,102.533333,0.868750,85.416667,4.073333,140.666667,10.066667,50.407558,163.98875,18.5480,114.805556,71.847222


In [48]:
print(icu.shape)
print(icu.columns)
missing_values_prop = icu.isnull().mean()
a = missing_values_prop*100.0
print("missings proportion:\n", a)
icu.head()

(10969, 25)
Index(['subject_id', 'gender', 'anchor_age', 'label', 'Anion_Gap_mean',
       'Bicarbonate_mean', 'Chloride_mean', 'Creatinine_mean', 'Glucose_mean',
       'Potassium_mean', 'Sodium_mean', 'Urea_Nitrogen_mean', 'weight_kg_mean',
       'height_cm_mean', 'bmi_mean', 'bp_sys_mean', 'bp_dia_mean', 'egfr_mean',
       'Heart_Rate_mean', 'Non_Invasive_Blood_Pressure_systolic_mean',
       'Non_Invasive_Blood_Pressure_diastolic_mean', 'Respiratory_Rate_mean',
       'Temperature_Celsius_mean', 'Admission_Weight_(Kg)_mean',
       'Height_(cm)_mean'],
      dtype='object')
missings proportion:
 subject_id                                     0.000000
gender                                         0.000000
anchor_age                                     0.000000
label                                          0.000000
Anion_Gap_mean                                 0.264381
Bicarbonate_mean                               0.264381
Chloride_mean                                  0.246148

,subject_id,gender,anchor_age,label,Anion_Gap_mean,Bicarbonate_mean,Chloride_mean,Creatinine_mean,Glucose_mean,Potassium_mean,...,bp_sys_mean,bp_dia_mean,egfr_mean,Heart_Rate_mean,Non_Invasive_Blood_Pressure_systolic_mean,Non_Invasive_Blood_Pressure_diastolic_mean,Respiratory_Rate_mean,Temperature_Celsius_mean,Admission_Weight_(Kg)_mean,Height_(cm)_mean
0,10000032,F,52,1,11.222222,25.388889,97.047619,0.466667,106.176471,5.100000,...,106.166667,64.666667,NaN,96.500000,88.900000,54.100000,20.700000,NaN,39.4,152.0
1,10000690,F,86,1,12.512821,30.487179,97.333333,0.940000,92.459459,4.256098,...,135.750000,63.000000,NaN,84.072917,122.893617,60.361702,22.557895,NaN,55.3,NaN
2,10000980,F,73,0,17.032967,22.857143,106.419355,2.398936,143.968750,4.450000,...,143.129870,78.467532,NaN,73.636364,142.454545,83.272727,20.545455,NaN,76.2,NaN
3,10001217,F,55,1,13.545455,26.636364,103.909091,0.491667,96.363636,4.109091,...,128.000000,86.000000,NaN,86.711538,126.900000,77.720000,19.038462,NaN,73.0,NaN
4,10001725,F,46,0,15.220000,25.780000,100.549020,0.820000,118.333333,4.305556,...,121.038217,75.566879,NaN,79.156250,100.406250,61.187500,17.531250,NaN,72.2,157.0
